# Start coding and Enjoy the journey

## Install the repo

In [ ]:
!git clone --recursive -b Nicola --single-branch https://github.com/nikcian/Visual-Place-Recognition-Project.git

In [ ]:
%cd Visual-Place-Recognition-Project
!git submodule update --init --recursive

## Install dependencies

In [ ]:
%cd image-matching-models

# 1. Install stable PyTorch (faster than compiling extensions)
# We force a stable version to access pre-built wheels
!pip install torch==2.5.1 torchvision==0.20.1 --index-url https://download.pytorch.org/whl/cu124

import torch
print(f"Switched to stable Torch version: {torch.__version__}")

# 2. Install 'uv' for speed
!pip install uv

# 3. Install PyG dependencies using pre-built wheels (Super fast, no compilation)
wheel_url = "https://data.pyg.org/whl/torch-2.5.1+cu124.html"
print(f"Installing binary wheels from: {wheel_url}")
!uv pip install --system torch-scatter torch-sparse torch-cluster torch-spline-conv -f {wheel_url}

# 4. Install the rest of the repository
!uv pip install --system -e .[all]

In [ ]:
!pip install faiss-cpu

## Download Datasets

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import shutil
import os

# CONFIGURAZIONE
# Cartella su Drive dove hai caricato i file zip
drive_folder = '/content/drive/MyDrive/data'

# Lista dei file zip da estrarre
# ⚠️ Sostituisci 'nome_quarto_file.zip' con il nome reale del tuo file
zip_files = [
    'tokyo_xs.zip',
    'sf_xs.zip',
    'gsv_xs.zip',
    'svox.zip'
]

# Percorso di destinazione
dest_dir = '/content/Visual-Place-Recognition-Project/data'
os.makedirs(dest_dir, exist_ok=True)

print(f"Inizio estrazione di {len(zip_files)} file da '{drive_folder}' a '{dest_dir}'...\n")

for zip_name in zip_files:
    source_path = os.path.join(drive_folder, zip_name)

    if os.path.exists(source_path):
        print(f"📂 Estrazione di {zip_name}...")
        try:
            shutil.unpack_archive(source_path, dest_dir)
            print(f"✅ {zip_name} estratto correttamente.")
        except Exception as e:
            print(f"❌ Errore durante l'estrazione di {zip_name}: {e}")
    else:
        print(f"⚠️  File non trovato: {source_path} (Saltato)")

print("\nOperazione terminata.")

## Run your First VPR Evalutation



In [ ]:
# Scegli il metodo (es. 'megaloc')
METHOD = 'megaloc'

print(f"🚀 Avvio VPR Evaluation con: {METHOD}")

# Usiamo il percorso assoluto per evitare errori se si riesegue la cella
%cd /content/Visual-Place-Recognition-Project

!python VPR-methods-evaluation/main.py \
--num_workers 8 \
--batch_size 32 \
--log_dir log_dir \
--method="$METHOD" \
--image_size 512 512 \
--database_folder 'data/tokyo_xs/test/database' \
--queries_folder 'data/tokyo_xs/test/queries' \
--num_preds_to_save 20 \
--recall_values 1 5 10 20 \
--save_for_uncertainty

print(f"\n✅ VPR evaluation completed")

In [ ]:
import os
import glob

def get_latest_experiment_paths(log_base_dir):
    """
    Versione corretta: Trova l'esperimento più recente E la cartella inliers più recente al suo interno.
    """
    if not os.path.exists(log_base_dir):
        print(f"❌ La cartella base non esiste: {log_base_dir}")
        return None, None, None, None

    # 1. Trova l'esperimento più recente
    subdirs = [os.path.join(log_base_dir, d) for d in os.listdir(log_base_dir) if os.path.isdir(os.path.join(log_base_dir, d))]
    if not subdirs:
        return None, None, None, None

    latest_exp = max(subdirs, key=os.path.getmtime)
    print(f"📂 Ultimo esperimento: {latest_exp}")

    preds_dir = os.path.join(latest_exp, 'preds')
    z_data_path = os.path.join(latest_exp, 'z_data.torch')

    # 2. Trova le cartelle preds_* (escluso 'preds')
    candidates = [d for d in glob.glob(os.path.join(latest_exp, 'preds_*')) if os.path.basename(d) != 'preds']

    if candidates:
        # --- CORREZIONE: Ordina per data di modifica (dal più recente) ---
        inliers_dir = max(candidates, key=os.path.getmtime)
        print(f"   ✅ Inliers dir più recente trovata: {os.path.basename(inliers_dir)}")
    else:
        inliers_dir = None

    return latest_exp, preds_dir, inliers_dir, z_data_path

# --- CONFIGURAZIONE ---
LOG_BASE_DIR = 'logs/log_dir/'

# Aggiorniamo subito le variabili con la funzione corretta
LATEST_EXP, PREDS_DIR, INLIERS_DIR, Z_DATA_PATH = get_latest_experiment_paths(LOG_BASE_DIR)

# Verifica output
if LATEST_EXP:
    print(f"✅ Preds Dir:    {PREDS_DIR}")
    print(f"✅ Inliers Dir:  {INLIERS_DIR}")
    print(f"✅ Z-Data Path:  {Z_DATA_PATH}")
else:
    print("⚠️ Impossibile configurare i percorsi automaticamente.")

## Run Image Matching on Retrieval Results



In [ ]:
# Esegue il matching sulle predizioni trovate

# Scegli il matcher (es. 'loftr', 'superglue', 'superpoint-lg', 'dedode')
MATCHER = 'loftr'  # <--- Cambia questo nome per provare altri metodi

print(f"🚀 Avvio re-ranking con: {MATCHER}")

if PREDS_DIR:
    print(f"✅ Cartella predictions rilevata: {PREDS_DIR}")

    !python match_queries_preds.py \
    --preds-dir "$PREDS_DIR" \
    --matcher "$MATCHER" \
    --device 'cuda' \
    --num-preds 20

    # --- AGGIORNAMENTO VARIABILI ---
    # La cartella 'inliers' è stata appena creata, quindi aggiorniamo i percorsi
    print("\n🔄 Aggiornamento variabili post-matching...")
    LATEST_EXP, PREDS_DIR, INLIERS_DIR, Z_DATA_PATH = get_latest_experiment_paths(LOG_BASE_DIR)

    if INLIERS_DIR:
        print(f"\n✅ Matching completato. Cartella inliers: {INLIERS_DIR}")
    else:
        print("⚠️ Attenzione: Inliers Dir non trovata dopo l'esecuzione.")
else:
    print("⚠️ Impossibile procedere: PREDS_DIR non definito.")

## Check Re-ranking Performance

In [ ]:
# Esegue il re-ranking usando i match geometrici
if PREDS_DIR and INLIERS_DIR:
    print(f"✅ Preds: {PREDS_DIR}")
    print(f"✅ Inliers/Reranked: {INLIERS_DIR}")

    print("📊 Calcolo nuove metriche di performance...")

    !python reranking.py \
    --preds-dir "$PREDS_DIR" \
    --inliers-dir "$INLIERS_DIR" \
    --num-preds 20 \
    --recall-values 1 5 10 20
else:
    print("⚠️ Percorsi mancanti.")

## Perform Uncertainty Evalutation

In [ ]:
# Valutazione dell'incertezza
if PREDS_DIR and INLIERS_DIR and Z_DATA_PATH:
    print(f"✅ Preds: {PREDS_DIR}")
    print(f"✅ Inliers: {INLIERS_DIR}")
    print(f"✅ Z-Data: {Z_DATA_PATH}")

    # Eseguiamo il modulo vpr_uncertainty
    !python -m vpr_uncertainty.eval \
    --preds-dir "$PREDS_DIR" \
    --inliers-dir "$INLIERS_DIR" \
    --z-data-path "$Z_DATA_PATH"
else:
    print("⚠️ Dati mancanti per la valutazione incertezza. Controlla la cella di setup.")